In [4]:
#Importing all Dependencies

import pandas as pd
import numpy as np
import glob
import os
import pandas_profiling as pp
import seaborn as sns
#sns.set(color_codes=True)
import matplotlib.pyplot as plt
%matplotlib inline
#import csv
import warnings
warnings.filterwarnings('ignore')
import sklearn
import scipy
#import random
#import statsmodels.api as sm
# from sklearn.preprocessing import StandardScaler
# from sklearn.neighbors import LocalOutlierFactor
# from sklearn.covariance import EllipticEnvelope
# from sklearn.ensemble import IsolationForest
# from sklearn import linear_model
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn import metrics
# from sklearn.decomposition import PCA
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import ParameterGrid
# from sklearn.grid_search import GridSearchCV
# from sklearn.model_selection import GridSearchCV
# from sklearn.ensemble import BaggingClassifier
# from sklearn.metrics import classification_report, f1_score, accuracy_score, precision_score, confusion_matrix
# from sklearn.pipeline import Pipeline

In [5]:
#### Importing the Loan Data of Interest

#columns of interest
headers = ['issue_d','loan_amnt','open_acc','acc_now_delinq','chargeoff_within_12_mths','delinq_2yrs','inq_last_6mths','num_accts_ever_120_pd','pub_rec','recoveries','grade','dti','annual_inc','dti_joint','annual_inc_joint','revol_bal','revol_bal_joint','debt_settlement_flag','num_actv_bc_tl','emp_length','total_il_high_credit_limit','tax_liens','total_bal_ex_mort','verification_status','verification_status_joint','purpose','revol_util','term']

# Compile the files
# skip rows that included text or footers
filenames = glob.glob(r"C:\Users\BieniekJon\Desktop\900\Mittal\Lending Club\Refresh\Lending_Club_Project\*.csv")
files = [pd.read_csv(filename, skiprows=1, skipfooter=2, usecols=headers) for filename in filenames]
lending_club = pd.concat(files, ignore_index=True)

In [37]:
#### Label each row as 'pre' or 'post' IPO

# convert date column to datetime
lending_club['issue_d'] = pd.to_datetime(lending_club['issue_d'])

# create label column
lending_club['time'] = np.where(lending_club['issue_d'].dt.year == 2014, 'pre', 'post')

# drop the date column as it creates noise
lending_club = lending_club.drop(['issue_d'], axis = 1)

['loan_amnt',
 'term',
 'grade',
 'emp_length',
 'annual_inc',
 'verification_status',
 'purpose',
 'dti',
 'delinq_2yrs',
 'inq_last_6mths',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'recoveries',
 'annual_inc_joint',
 'dti_joint',
 'verification_status_joint',
 'acc_now_delinq',
 'chargeoff_within_12_mths',
 'num_accts_ever_120_pd',
 'num_actv_bc_tl',
 'tax_liens',
 'total_bal_ex_mort',
 'total_il_high_credit_limit',
 'revol_bal_joint',
 'debt_settlement_flag',
 'time']

In [38]:
#### Featue Engineering

# combine columns for like features that were broken into joint and individual
lending_club["new dti"] = np.where(lending_club["dti_joint"].isnull(), lending_club["dti"], lending_club["dti_joint"])
lending_club["new annual_inc"] = np.where(lending_club["annual_inc_joint"].isnull(), lending_club["annual_inc"] , lending_club["annual_inc_joint"])
lending_club["new revol_bal"] = np.where(lending_club["revol_bal_joint"].isnull(), lending_club["revol_bal"] , lending_club["revol_bal_joint"])
lending_club['new verification'] =  np.where(lending_club["verification_status_joint"].isnull(), lending_club["verification_status"], lending_club["verification_status_joint"])

# convert text columns to numeric
lending_club["new debt flag"] = np.where(lending_club["debt_settlement_flag"] == "N", 0,1)

# combine employment length columns into one
lending_club["emp_length_new"] = np.where(lending_club['emp_length']=="10+ years", "10+ years", 
                                      np.where(np.isin(lending_club['emp_length'], ["< 1 year", "1 year",'2 years','3 years','4 years','5 years']), "1-5 years", 
                                               np.where(np.isin(lending_club['emp_length'], ['6 years','7 years','8 years','9 years']), "6-9 years", "n/a")))
# combine purpose of loans columns into one column 
lending_club["new purpose"] = np.where(lending_club['purpose']=="debt_consolidation", "debt consolidation", np.where(lending_club['purpose']=="credit_card", "credit card", "other"))

# convert loan grades to numeric. A or B = 1, remaining = 0
lending_club['new grade'] = np.where(np.isin(lending_club['grade'], ["A", "B"]), 1 , 0)


In [45]:
# Creat dummy variables for categorical features
cols_to_transform = [ 'emp_length_new', 'new verification', 'term', 'new purpose']

lc_dumvar = pd.get_dummies(lending_club,  columns = cols_to_transform, drop_first = False)

(535074, 44)

In [49]:
# remove columns that are no longer needed after condensing in previous step
remove = ['emp_length','annual_inc','dti','annual_inc_joint','dti_joint','revol_bal','revol_bal_joint','debt_settlement_flag', 'verification_status','verification_status_joint', 'purpose']

lc_fin_col = lc_dumvar.drop(remove, axis='columns')

In [73]:
# drop rows with NAs after investgating
lc_fin_shape = lc_fin_col.dropna()

# reset index
lc_fine_struct = lc_fin_shape.reset_index(drop=True)